In [1]:
from flask import Flask, render_template, request

import numpy as np
import os

import cv2 as cv2
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model



filepath = 'E:/KJSCE_MP/Backup]/MiniProject/model.h5'
model = load_model(filepath)
print(model)

print("Model Loaded Successfully")



def pred_maize_disease(img_maize_crop):

    image_process = load_img(img_maize_crop, target_size = (408,615))
    print("Image Recieved")

    image_process = img_to_array(image_process)/255
    image_process = np.expand_dims(image_process, axis=0)
    
    result = model.predict(image_process)

    print("Result ",result)

    pred = np.argmax(result, axis=1)

    print(pred)




    if(pred==0):
        return "Maize - Blight",'Maize - Blight.html'

    elif(pred==1):
        return "Maize - Common Rust",'Maize - Commonrust.html'

    elif(pred==2):
        return "Maize - Gray Leaf Spot",'Maize - Grayspots.html'
        
    elif(pred==3):
        return "Maize - Healthy",'Maize - Healthy.html'




def init_grabcut_mask(h, w):
            mask = np.ones((h, w), np.uint8) * cv2.GC_PR_BGD #Possible background Pixel
            mask[h//4:3*h//4, w//4:3*w//4] = cv2.GC_PR_FGD   #Possible Foreground Pixel
            mask[2*h//5:3*h//5, 2*w//5:3*w//5] = cv2.GC_FGD  #Obvious Foreground Pixel
            return mask

# def add_contours(image, mask):

#     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#     if len(contours) != 0:
    
#         cv2.drawContours(image, contours, -1, (255, 0, 0), 3)
#         c = max(contours, key = cv2.contourArea)
#         x,y,w,h = cv2.boundingRect(c)
#         cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0) ,2)
            


def remove_background(image):
            
    h, w = image.shape[:2]
    mask = init_grabcut_mask(h, w)
    bgm = np.zeros((1, 65), np.float64)
    fgm = np.zeros((1, 65), np.float64)
    cv2.grabCut(image, mask, None, bgm, fgm, 1, cv2.GC_INIT_WITH_MASK)
    mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    result = cv2.bitwise_and(image, image, mask = mask_binary)
    # add_contours(result, mask_binary) # Visualizations
    return result






app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def home():
        return render_template('index.html')
    
 

@app.route("/predict", methods = ['GET','POST'])
def predict():

     if request.method == 'POST':
        file = request.files['image']
        filename = file.filename        
        print("Input posted = ", filename)

        # print("Jeu")

                
        file_path = os.path.join('E:/KJSCE_MP/Backup]/FinalBuildv2.0/assets/upload', filename)
        file.save(file_path)




        img = cv2.imread(file_path)
        h,w = img.shape[:-1]

        # print(h,w)

        init_grabcut_mask(h,w)

        plt.imshow(init_grabcut_mask(h, w))
        
        im = cv2.resize(img,(3*205, 3*136))
        im = remove_background(im)

        cv2.imwrite(file_path,im)


        

        print("Predicting ")
        pred, output_page = pred_maize_disease(img_maize_crop=file_path)
              
        return render_template(output_page,pred_output = pred, user_image = file_path)
    
if __name__ == "__main__":
    app.run(threaded=False,port=8080) 

OSError: No file or directory found at E:/KJSCE_MP/Backup]/MiniProject/backend_model.h5